**<p style="color:red;">If you are using Binder instead of downloading the repository and working locally, this script might not run due to the relatively large datasets and Binder's limited resources. I recommend working locally on your own computer for optimal performance.</p>**



# Transitioning from Surface Data to Vertical Profiles

In previous sessions, we focused on surface data, particularly wind and sea surface temperature (SST). Now, we transition to analyzing vertical profiles using Argo data, which provide temperature and  salinity and their uncertainty at various depths. Unlike the SST data from Session 2, which was accessed via OPeNDAP, here we work with objectively mapped Argo data, downloaded over HTTPS. We will download, process, and combine these datasets, incorporating error handling and logical conditions such as if-else statements to ensure robustness.


[Argo floats](https://www.marine.ie/site-area/infrastructure-facilities/marine-research-infrastructures/argo-network): autonomous instruments that drift with ocean currents and periodically dive to various depths. On their way to the surface, they measure temperature, salinity, and depth. 


**<p style="color:royalblue;">A brief exploration of gridding techniques for in situ data can be found in the notebook `IntroGridding.ipynb`.</p>**

In this session, we will explore both horizontal and vertical patterns of temperature, salinity, and depth across different ocean basins. You will calculate and analyze mixed layer depth and calculate parameters averaged over the mixed layer and assess whether surface measurements can serve as reliable proxies for mixed layer processes.

Run the next cells.

In [ ]:
# import necessary libraries
import os
import requests 
import zipfile
import xarray as xr 
import numpy as np 
import gsw


# Visualization
import holoviews as hv
#from IPython.display import Image
import hvplot.xarray
hv.extension('bokeh')

import warnings
warnings.filterwarnings("ignore")



After importing the necessary libraries, we load the Argo data from EN4 using the HTTPS protocol. These datasets are available via the [Met Office website](https://www.metoffice.gov.uk/hadobs/en4/download-en4-2-2.html). We select the objectively analyzed data, which have been error-corrected and include uncertainty information. In the following code, we use a .txt file that contains the list of datasets we want to download. The code requests the datasets and uses if-else statements to manage the download process, ensuring that files are downloaded - if necessary, unzipped - , and processed. Additionally, the code now checks if the zip files already exist, in which case they are simply read in.

In [ ]:
# run the cell

# Define the folder where files will be saved and unzipped
save_directory = '../Data/EN4'
# creates the save_directory, if it not exists
os.makedirs(save_directory, exist_ok=True)

# Read the list of URLs from the download list file
with open('../Data/EN4/EN.4.2.2.analyses.g10.download-list.txt') as f:
    urls = f.readlines()

# List to store the paths of extracted NetCDF files
nc_files = []

# Loop through each URL, download, unzip, and collect NetCDF file paths
for url in urls:
    url = url.strip()  # Clean up the URL
    if url:  # Check if the URL is not empty
        file_name = url.split('/')[-1].strip()
        file_path = os.path.join(save_directory, file_name)
        
        # Check if the zip file already exists
        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
            print(f"{file_name} already exists. Skipping download.")
        else:
            # Download the file
            print(f"Downloading {file_name} to {file_path}...")
            r = requests.get(url)
        
            if r.status_code == 200: # sucessful request
                # Save the zip file
                with open(file_path, 'wb') as file:
                    file.write(r.content)
                
                # Check if the file exists and is not empty
                if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                    print(f"{file_name} downloaded successfully.")
                else:
                    print(f"Failed to download {file_name}, file not found or empty.")
                    continue
            else:
                print(f"Failed to download {file_name}, status code: {r.status_code}")
                continue
        
        # Unzip the file and collect NetCDF file paths
        print(f"Extracting {file_name}...")
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(save_directory)
            for extracted_file in zip_ref.namelist():
                if extracted_file.endswith('.nc'):
                    print(f"Found {extracted_file}")
                    nc_files.append(os.path.join(save_directory, extracted_file))

# Use open_mfdataset to combine all NetCDF files into a single dataset
if nc_files:
    ds_en4 = xr.open_mfdataset(nc_files, combine='by_coords')
    print(ds_en4)
else:
    print("No datasets downloaded or processed.")


In case the connection becomes overloaded because we're all trying to download the data at the same time, please download the data manually using the links in the txt file and store them in the folder [../Data/EN4](../Data/EN4/). Then, run the cell above again.

**1. Exercise**:

Explore the dataset you just loaded by answering the following questions:

    What is the shape of the dataset?
    Examine the dimensions and size of the dataset.

    Which variables are contained within the dataset?
    List the variables provided in the dataset and their descriptions.

    What is the resolution of the data?
    Analyze the spatial resolution of the dataset.

    What units are the data presented in?
    Investigate the units of measurement for the variables in the dataset.

    

Use the appropriate methods like e.g. .dims, .data_vars, and .attrs to extract this information.

In [ ]:
# your code here to get the dimensions


In [ ]:
# your code here to get the variables


In [ ]:
# your code to get the horizontal resolution of the data (hint: use np.diff)

Check the attributes of a specific variable, for example, temperature.  
Hint: Use `ds_en4['temperature'].attrs` to explore all attributes associated with this variable.

Identify the attribute that corresponds to the unit (e.g., `'units'`).

Use the `.get()` method to retrieve the value of the 'units' attribute:  
`ds_en4['temperature'].attrs.get('units')`

Consider writing a loop to retrieve the units for all variables in the dataset.  
You can iterate through ds_en4.data_vars and access the `.attrs.get('units')` for each variable.

In [ ]:
# your code to get the units of each variable 


**Question**: What is the vertical spacing between depth levels?

Examine the differences between consecutive depth levels to understand the vertical structure of the dataset.

Hint: You can again use `np.diff()` to calculate the differences between consecutive depth levels.

In [ ]:
# your code here



In the following, we plot the temperature.

### Plotting with hvplot:

We use hvplot because it allows us to explore multidimensional datasets interactively and efficiently. This is particularly useful for visualizing datasets with 3 or 4 dimensions (e.g., time, longitude, latitude, and depth). `hvplot` works directly with xarray, allowing the creation of interactive plots.

For instance, when we visualize temperature using the following command:

In [ ]:
# run the cell

ds_en4.temperature.hvplot('lon', 'lat', width=700, height=400)



**2. Exercise**: 

Convert the temperature from Kelvin to Celsius and then visualize it using `hvplot`.   
After converting the temperature, update the units in the dataset to reflect the change from Kelvin to Celsius. You can also customize the plot by setting color limits (`clim`) and choosing a different colormap.

In [ ]:
# your code and plot here


### Introduction to Depth Levels in EN4

Unlike the datasets we have explored previously, the EN4 dataset includes data at various depth levels. This allows us to examine how variables such as temperature and salinity change not just horizontally but also vertically throughout the ocean.

We create a vertical section plot. Specifically, we will plot a temperature section in the Pacific Ocean at 220°E (which is equivalent to 140°W).

In [ ]:
# run the cell
pacific_section = ds_en4.sel(lon = 220) # 220-360= -140 -> Pacific
pacific_section['depth'] = -1 * pacific_section['depth']  # Invert depth
pacific_section.temperature[:,:35,:].hvplot.quadmesh('lat', 'depth', clim=(0, 30), cmap='RdBu_r')

And the Atlantic at 30°W:

In [ ]:
# run the cell

atlantic_section = ds_en4.sel(lon = 330) #330-360 = 30°W -> Atlantic
atlantic_section['depth'] = -1 * atlantic_section['depth']  # Invert depth
atlantic_section.temperature[:,:35,:].hvplot.quadmesh('lat', 'depth', clim=(0, 30), cmap='RdBu_r')

We will modify the plot by using contour lines for visualization:

This makes it easier to visually identify patterns and gradients in the data.


In [ ]:
# ATLANTIC
# run the cell
#with contours
atlantic_section = ds_en4.sel(lon=330)  # 30°W
atlantic_section['depth'] = -1 * atlantic_section['depth']  # Invert depth

# Plot filled temperature contours
atlantic_section.temperature[:,:35,:].hvplot.contourf(
    x='lat', 
    y='depth', 
    levels=30,  # Number of contour levels
    cmap='RdBu_r',  # Reversed red-blue colormap
    clim=(-2, 30), 
    width=700, 
    height=400, 
    title='Atlantic Temperature Filled Contours at 30°W'
)

In [ ]:
# PACIFIC
# run the cell
# with contours 
# Plot filled temperature contours
pacific_section.temperature[:,:35,:].hvplot.contourf(
    x='lat', 
    y='depth', 
    levels=30,  # Number of contour levels
    cmap='RdBu_r',  # Reversed red-blue colormap
    clim=(-2, 30), 
    width=700, 
    height=400, 
    title='Pacific Temperature Filled Contours at 140°W'
)

### Your Observations

.....

**Extra Exercise:** Analyse Salinity

### Connecting Back to Session 2 and Introducing the Vertical Section - ENSO
In Session 2, we analyzed sea surface temperature (SST) data over an extended period and calculated both anomalies and ENSO composites. If you're not familiar with this, I recommend revisiting Session 2 and reviewing the exercises.

We observed that temperature anomalies during ENSO events exhibit significant amplitude in the central Pacific, particularly around the equator. 


To better understand this, let’s first take a look at the next diagram, which shows the variation of surface temperatures over time and longitude along the equator.

In [ ]:
# run the cell

enso_section = ds_en4.sel(lat =0,lon = slice(200,260)) #160°W-100°W
enso_section['depth'] = -1 * enso_section['depth']  # Invert depth (we need this later for the section plot)


# Select the first depth level for the Hovmöller plot
first_depth_level = enso_section.isel(depth=0)  # First available depth level

# Create a Hovmöller plot: time vs longitude at the equator for the first depth level

hovmoeller_plot = first_depth_level.temperature.hvplot.quadmesh('time', 'lon', 
                                                                           clim=(25, 30), cmap='coolwarm', 
                                                                           width=700, height=400, invert = True) # invert=True reverses the values along the y-axis (e.g., time runs from bottom to top)

# Display the plot
hovmoeller_plot

This plot is called a **Hovmöller** diagram an it is very often used in oceanography/climatology.  

In the Hovmöller diagram, which shows absolute temperatures at the surface along the equator, we can already observe differences in temperature across time and longitude. These differences could be linked to the El Niño-Southern Oscillation (ENSO) cycle.

Temperature Gradients: Notice how the western Pacific generally has warmer surface temperatures than the eastern Pacific. This persistent gradient reflects somehow the typical climate state of the tropical Pacific, where warm water piles up in the west due to the trade winds.

ENSO Influence: During El Niño events, the warm water that is usually confined to the western Pacific shifts eastward, resulting in warmer-than-usual temperatures across the central and eastern Pacific. Conversely, during La Niña events, cold upwelling in the eastern Pacific intensifies, leading to a stronger east-west temperature gradient with colder temperatures in the east and warmer temperatures in the west.



### But what happens below the surface? How do these patterns extend vertically through the water column?

In the following section, we will take a closer look at the vertical temperature structure along the central Pacific, from 160°W to 100°W, to explore how ENSO impacts not just the surface but the entire vertical profile of the ocean.

In [ ]:
# run the cell

enso_section.temperature[:,:20,:].hvplot.contourf(
    x='lon', 
    y='depth', 
    levels=10,  # Number of contour levels
    cmap='coolwarm',  # Reversed red-blue colormap
    clim=(10, 30), 
    width=700, 
    height=400, 
    title='Pacific Temperature Filled Contours at the equator'
)

### Explaining the Temperature Section and ENSO Influence

When plotting this temperature section along the equator, we can observe differences between the temperatre especially between 2022 and 2023. This region is strongly influenced by ENSO (El Niño-Southern Oscillation) events. During this period, several La Niña events were followed by an El Niño in 2023, and this is reflected in the temperature patterns.

One notable feature is that surface temperatures are generally higher in the western Pacific, while the thermocline (the boundary between warm surface water and colder deep water) is tilted. In the west, the warm water extends much deeper than in the east.

This tilting of the thermocline is a classic feature of ENSO. During La Niña events, the easterly trade winds strengthen, pushing water toward the western Pacific and deepening the thermocline there, while in the eastern Pacific, upwelling brings colder water to the surface. Conversely, during El Niño events, these trade winds weaken or reverse, causing the warm water to shift eastward and leads to higher surface temperatures in the central and eastern Pacific.

In summary, the tilting thermocline and the warm water depth differences could be related to the ENSO cycle, reflecting the dynamic interplay between ocean temperatures and atmospheric wind patterns in this region.

The following picture from [US National Weather Service](https://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensocycle/lanina_schem.shtml) briefly describes the La Nina phenomenon.

In [2]:
#run the cell

from IPython.display import Image

# How to display an image from a URL
Image(url='https://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensocycle/djfschem_lanina.gif', width=500, height=300)

# Density, Mixed Layer Depth, Mixed Layer Salinity and Temperature 

Temperature is a key factor in determining density, alongside with salinity and pressure. Warmer water is less dense than colder water, leading to a stratified water column. 

Next, we’ll explore how these temperature differences impact the density and MLD in the Pacific.

We will use the GSW (Gibbs SeaWater) library to calculate pressure, potential temperature, and potential density from the available temperature, depth, and salinity data. Further information: https://www.teos-10.org/pubs/gsw/html/gsw_contents.html

In [22]:
# run the cell 

# Calculate pressure based on depth
depth = ds_en4['depth']
lat = ds_en4['lat']
pressure = gsw.p_from_z(-depth, lat) # p varies with depth and geographical location

# Calculate potential temperature ( temp of a water parcel when its moved adiabatically to the surface pressure) and density
temp = ds_en4['temperature']
sal = ds_en4['salinity'] 
# potential temperature:
ptemp = gsw.pt0_from_t(sal, temp, pressure) 

# potential density:
sigma0 = gsw.sigma0(sal, ptemp)

# Set correct attributes for the potential density
sigma0.name = 'Potential Density'
sigma0.attrs['long_name'] = 'Potential Density (Sigma_0)'
sigma0.attrs['units'] = 'kg m-3'

# Remove unwanted attributes that come from the original salinity data (pop is used to remove specific attributes)
sigma0.attrs.pop('valid_min', None)
sigma0.attrs.pop('valid_max', None)
sigma0.attrs.pop('standard_name', None)

# Set correct attributes for the potential temperature

ptemp.name = 'Potential Temperature'
ptemp.attrs['long_name'] = 'Potential Temperature'
ptemp.attrs['units'] = '°C'

# Remove unwanted attributes that come from the original salinity data
ptemp.attrs.pop('valid_min', None)
ptemp.attrs.pop('valid_max', None)
ptemp.attrs.pop('standard_name', None)


# add variables to dataset
ds_en4['ptemp'] = ptemp
ds_en4['sigma0'] = sigma0

We plot the density section at the same location as the previous temperature section at the equator. When viewing the density section, corresponding patterns to temperature become apparent, indicating that temperature changes could have a dominant impact on density variations in the ENSO region.

In [ ]:
# run the cell to see the plot
sigma_enso = sigma0.sel(lat =0,lon = slice(200,260)) #160°W-100°W
sigma_enso['depth'] = -1 * sigma_enso['depth']  # Invert depth

# plot the first 20 depth levels (you can experiment with the number of depth levels)
sigma_enso[:,:20,:].hvplot.contourf(
    x='lon', 
    y='depth', 
    levels=20,  # Number of contour levels
    cmap='coolwarm',  # Reversed red-blue colormap
    clim=(21, 27), 
    width=700, 
    height=400, 
    title='Pacific Density Filled Contours at the equator'
)

## Mixed Layer Depth (MLD)

The Mixed Layer (ML) is the uppermost layer of the ocean, where properties like temperature and salinity are relatively uniform due to mixing processes etc. The MLD represents the depth at which the mixed layer transitions to the more stratified layer below, typically marked by a significant change in density, temperature, or salinity.

ENSO events can cause the MLD to shift, as the position of the pycnocline (which is where the water density changes rapidly with depth) changes in response to e.g. warming or cooling of the surface waters.


For the MLD, we use a density threshold, which calculates the MLD based on a change in potential density (0.03 kg m-3) starting from 5 meters depth (the first depth layer). The mixed layer facilitates energy transfer from the air to the water and is also a zone where biological activity is generally enhanced.

In [24]:
# run the cell

# select density of the uppermost layer as reference
# We use isel to select by the index of a coordinate (e.g., depth=0 corresponds to the uppermost layer).
# Otherwise use sel when selecting based on the exact value of a coordinate (e.g., depth=5 for 5 m).
surface_density = sigma0.isel(depth=0)

# Calculate the density difference between the surface and each depth level
density_diff = sigma0 - surface_density

# Define a density threshold for MLD calculation (0.03 kg/m³)
density_threshold = 0.03

# Create a mask that is True (or 1) where density difference exceeds the threshold
mask = density_diff >= density_threshold

# Use the mask to find the minimum depth where the density difference meets the threshold, indicating the mld
mld_from_surface_difference = depth.where(mask).min(dim='depth')

mld_from_surface_difference.attrs['long_name'] = 'mixed layer depth based on the surface difference (threshold 0.03 kg m-3)'
mld_from_surface_difference.attrs['units'] = 'm'
# Add Mixed Layer Depth to the dataset
ds_en4['mld'] = mld_from_surface_difference


**3. Exercise: Plot the mean MLD over time. What do you observe?**

Use `hvplot()`  for visualization.

In [1]:
# your code here 

### Annual Amplitude and internannual variability 

**4. Exercise**:  
Now we are interested in investigating where the mld exhibits a strong seasonal cycle and where interannual variability is more dominant.

Your task is to first calculate the annual amplitude based on the climatological seasonal cycle and visualize it.

Any ideas how we calculate the annual amplitude based on the climatology?

Hints are given in the following code cell

**<p style="color:royalblue;">Given that we only have four years of data, the results may not be fully representative, but they can still provide useful insights. Think about how you would approach the calculation of the seasonal amplitude and how you would assess the interannual variability. Consider removing the seasonal cycle to isolate interannual fluctuations.</p>**

In [ ]:
# your code here to calculate the annual amplitude of MLD

# Group data by month
# monthly_climatology =

# 
# Calculate the amplitude as the difference between the maximum and minimum monthly_climatology devided by 2
# annual_amplitude = 


# Plot the amplitude 



**Removing Seasonal Variations to Analyze Remaining Variability**

Now, remove the seasonal variations from the MLD data in order to focus on the remaining interannual variability. Even though we are working with only four years of data and not applying a detrending approach, this method helps isolate fluctuations beyond the regular seasonal cycle, providing insights into longer-term variability.

In [ ]:
# your code here to calculate the interannual variability
# Remove the seasonal cycle to calculate anomalies, you already calculated the monthly_climatology....
#anomalies = 



# Calculate the standard deviation of the anomalies (interannual variability)
# interannual_variability = 


# Plot interannual variability




**Q: What do you observe?**  
Where does the seasonal cycle dominate, and where are interannual changes pronounced?


**<p style="color:royalblue;">Keep in mind that we are analyzing only four years of data; longer time series would be better suited for analyzing interannual variability in detail.</p>**


**5. Exercise**: 
1. Calculate the annual amplitude of your dataset `ds_en4` from the first depth layer and plot salinity. 
2. Average the salinity and temperature within the mixed layer. Then, calculate and plot the annual amplitude of the mixed layer salinity and temperature.

In [ ]:
# your code here to calculate the annual amplitude of ds_en4 and plot salinity



In [ ]:
#Optional: Plot the annual amplitude of temperature


Average the `ds_en4` variables within the MLD, then calculate the annual amplitude and plot salinity.

Tip: Use a mask to select only the data within the Mixed Layer.

In [ ]:
# Calculate mixed layer salinity by averaging salinity up to the mixed layer depth (mld)

# Hint: Mask values deeper than the mixed layer depth (mld) using .where()  and replace ...
# ds_mld_masked = ds_en4.where(...)

# Compute the mean within the mixed layer  (its a mean over depth)
# mixed_layer_ds = 

# Group the mixed layer variables by month and compute the monthly climatology
# This you know already from SST data 
#monthly_climatology = 

# Calculate the annual amplitude as the difference between the maximum and minimum monthly climatology divided by 2
# This you have done before, scroll up and maybe copy relevant code lines
# annual_amplitude_ds_mld =

# Plot the annual amplitude of mixed layer salinity using hvplot
#annual_amplitude_ds_mld['salinity'].hvplot(clim=(0, 1), cmap='plasma')



**Debugging**:  
Consider how you can confirm that only the data within the mixed layer depth (MLD) are included in the calculation. To verify this, you could plot the masked salinity data to visually check whether the masking based on the MLD was applied correctly. This can be useful for debugging and ensuring that only the intended data are used.

In [ ]:
# run the cell, check if the variable name is correct

ds_mld_masked['salinity'].hvplot('lon', 'lat', cmap='viridis', width=700, height=400)

In [ ]:
# or use plot() and choose different time and depth

ds_mld_masked['salinity'][1,3,:,:].plot()

**Final Exercise** (for this session):

Calculate and Plot the RMSE:

- Calculate the RMSE between salinity in the first layer and the mixed layer salinity.
- Repeat the same for temperature.
- Plot both RMSE values on separate plots.

The Root Mean Square Error (RMSE) is calculated using the following formula:
$$
\text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (x_i - y_i)^2}
$$
​

Where:

$x_i$​ are the values from the first layer (e.g. salinity or temperature).  
$y_i​$ are the values from the mixed layer.  
$N$ is the number of data points.

**<p style="color:royalblue;">RMSE is typically used to measure the differences between predicted and observed (or true) values. In this case, we are using it to compare surface values (first layer) with mixed layer values, helping us assess the variability between these two layers.</p>**

In [ ]:
# your code here
# Calculate RMSE between variable at surface and variable averaged in the Mixed Layer

# Compute the difference between your variables in the first layer and mixed layer averaged variables at each time step
difference = 

# Square the differences
squared_difference = 

# Compute the mean of the squared differences over time
mean_squared_difference = 

# Take the square root to get the RMSE
rmse = 

# Plot the RMSE to visualize where salinity is a better or worse proxy for mixed layer salinity
rmse['salinity'].hvplot(clim=(0, .5), cmap='inferno', title='RMSE between Salinity and Mixed Layer Salinity')


In [ ]:
# plot rmse of ptemp

rmse['ptemp'].hvplot(clim=(0, .5), cmap='inferno', title='RMSE between Temperature and Mixed Layer Temperature')

RMSE provides a measure of the average difference between the values in the first layer and the mixed layer. A higher RMSE indicates greater variability or inconsistency between the two layers, suggesting that conditions in the first layer and the mixed layer differ significantly. Conversely, a lower RMSE indicates that the first layer and mixed layer have similar properties, meaning that, for example, salinity in the first layer is a good proxy for the mixed layer.

This is particularly interesting because we often rely on satellite data due to their high spatial and temporal resolution, whereas in-situ data typically have more limited coverage. Satellites allow us to rapidly obtain global datasets, but the challenge is to determine whether surface measurements are sufficient to accurately represent the processes occurring in the mixed layer—the layer that is in direct contact and exchange with the atmosphere. Understanding this can help improve our ability to use surface satellite observations to infer subsurface processes in the ocean's mixed layer.



In [33]:
#run the cell to save the uppermost variables from ds_en4

ds_en4[['salinity','sigma0','mld']].isel(depth=0).to_netcdf('../Data/EN4/EN4_uppermost_2020_2023.nc', engine='h5netcdf', compute=True)


In [38]:
# close the dataset and remove all .nc files from the EN4 folder to save space
ds_en4.close()
for file in nc_files:
    os.remove(file)